In [ ]:
pip install python-chess

In [ ]:
import pandas as pd
import numpy as np
import chess.pgn
import chess
from random import randint
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, callbacks, losses

In [ ]:
siamese_model = tf.keras.models.load_model("/kaggle/input/unsupervised-siamese-100000-dataset/unsupervised_siamese_100000-100000_V1.h5")

In [ ]:
def fen2bitstring(fen):
    board = chess.Board(fen)
    bit_string = np.zeros(773, dtype = np.int8)
    
    def piece_index(piece_type):
        if piece_type == chess.PAWN:
            return 0
        elif piece_type == chess.KNIGHT:
            return 1
        elif piece_type == chess.BISHOP:
            return 2
        elif piece_type == chess.ROOK:
            return 3
        elif piece_type == chess.QUEEN:
            return 4
        elif piece_type == chess.KING:
            return 5
        else:
            return -1
    
    for square in chess.SQUARES:
        piece = board.piece_at(square)
        if piece is not None:
            bit_index = 2 * 6 * square
            bit_string[bit_index] = int(piece.color)  #side (White: 1, Black: 0)
            bit_index += 1 + 6 * piece_index(piece)  #piece type (pawn: 0, knight: 1, bishop: 2, rook: 3, queen: 4, king: 5)
            bit_string[bit_index] = 1
            #We have encode 768 bits
    bit_string[768] = int(board.turn)
    bit_string[769] = int(board.has_kingside_castling_rights(chess.WHITE))
    bit_string[770] = int(board.has_queenside_castling_rights(chess.WHITE))
    bit_string[771] = int(board.has_kingside_castling_rights(chess.BLACK))
    bit_string[772] = int(board.has_queenside_castling_rights(chess.BLACK))
    
    return bit_string

In [ ]:
def randPosiGeneratorFromGame(game):
    random_positions_array = []
    positions = []
    board = game.board()

    for move in game.mainline_moves():
        if not board.is_capture(move):
            position = board.fen()
            bitstring_position = fen2bitstring(position)
            positions.append(bitstring_position)

        board.push(move)

        no_capture_posis = len(positions)
    if no_capture_posis > 5:
        for i in range(10):
            random_index = randint(5, no_capture_posis - 1)
            random_positions_array.append(positions[random_index])


    return random_positions_array

In [ ]:
pgn_file = open("../input/ccrl-dataset/CCRLdb.pgn")

random_positions_white_win_bitstring = []
random_positions_black_win_bitstring = []

i = 0
while i < (200000 + 200000):
    if i > 200000:
        game = chess.pgn.read_game(pgn_file)
        if game.headers["Result"] == "1-0":
            temp1 = randPosiGeneratorFromGame(game)
            random_positions_white_win_bitstring += temp1
        elif game.headers["Result"] == "0-1":
            temp2 = randPosiGeneratorFromGame(game)
            random_positions_black_win_bitstring += temp2
        if game is None:
            break
    i = i + 1
        
pgn_file.close()

In [ ]:
len(random_positions_white_win_bitstring)

In [ ]:
win = [1]
white_win_labels = []
for i in range(len(random_positions_white_win_bitstring)):
    white_win_labels.append(win)
loss = [0]
black_win_labels = []
for i in range(len(random_positions_black_win_bitstring)):
    black_win_labels.append(loss)

In [ ]:
white_win_bitstring_tensor = tf.convert_to_tensor(random_positions_white_win_bitstring, dtype=tf.float32)

white_win_labels_tensor = tf.convert_to_tensor(white_win_labels, dtype=tf.int32)

black_win_bitstring_tensor = tf.convert_to_tensor(random_positions_black_win_bitstring, dtype=tf.float32)

black_win_labels_tensor = tf.convert_to_tensor(black_win_labels, dtype=tf.int32)

In [ ]:
white_win_dataset = tf.data.Dataset.from_tensor_slices(white_win_bitstring_tensor)
black_win_dataset = tf.data.Dataset.from_tensor_slices(black_win_bitstring_tensor)

white_win_labels_dataset = tf.data.Dataset.from_tensor_slices(white_win_labels_tensor)
black_win_labels_dataset = tf.data.Dataset.from_tensor_slices(black_win_labels_tensor)

In [ ]:
paired_dataset = tf.data.Dataset.zip((white_win_dataset, black_win_dataset))
paired_labels_dataset = tf.data.Dataset.zip((white_win_labels_dataset, black_win_labels_dataset))

In [ ]:
'''trained_pos2vec_weights = np.load('/kaggle/input/100000-weights-pos2vec/trained_pos2vec_weights_100000.npy', allow_pickle = True)
weights_encoder1 = trained_pos2vec_weights[0]
weights_encoder2 = trained_pos2vec_weights[1]
weights_encoder3 = trained_pos2vec_weights[2]
weights_encoder4 = trained_pos2vec_weights[3]'''

In [ ]:
'''# Weights are taken after training on pos2vec first
def create_pos2vec_model():
    dbn_model = tf.keras.Sequential()

    layer1 = tf.keras.Sequential()
    encoder1 = layers.Dense(600, activation='relu', input_shape=(773,))
    layer1.add(encoder1)
    dbn_model.add(layer1)

    layer2 = tf.keras.Sequential()
    encoder2 = layers.Dense(400, activation='relu', input_shape=(600,))
    layer2.add(encoder2)
    dbn_model.add(layer2)

    layer3 = tf.keras.Sequential()
    encoder3 = layers.Dense(200, activation='relu', input_shape=(400,))
    layer3.add(encoder3)
    dbn_model.add(layer3)

    layer4 = tf.keras.Sequential()
    encoder4 = layers.Dense(100, activation='relu', input_shape=(200,))
    layer4.add(encoder4)
    dbn_model.add(layer4)
    
    # Output Layer: 100 units
    output_layer = layers.Dense(100, activation='relu')
    dbn_model.add(output_layer)

    # Set the weights for the DBN layers
    dbn_model.layers[0].layers[0].set_weights(weights_encoder1)  # First encoder layer
    dbn_model.layers[1].layers[0].set_weights(weights_encoder2)  # Second encoder layer
    dbn_model.layers[2].layers[0].set_weights(weights_encoder3)  # Third encoder layer
    dbn_model.layers[3].layers[0].set_weights(weights_encoder4)  # Fourth encoder layer

    return dbn_model'''

In [ ]:
'''def create_siamese_network():
    pos2vec_model1 = create_pos2vec_model()
    pos2vec_model2 = create_pos2vec_model()
    
    input1 = layers.Input(shape = (773,))
    input2 = layers.Input(shape = (773,))
    
    features1 = pos2vec_model1(input1)
    features2 = pos2vec_model2(input2)
    
    concatenated_features = layers.Concatenate()([features1, features2])
    
    fc1 = layers.Dense(400, activation = 'relu')(concatenated_features)
    fc2 = layers.Dense(200, activation = 'relu')(fc1)
    fc3 = layers.Dense(100, activation = 'relu')(fc2)
    output_layer = layers.Dense(2, activation = 'softmax')(fc3)
    
    siamese_model = tf.keras.Model(inputs=[input1, input2], outputs=output_layer)
    
    return siamese_model'''

In [ ]:
from sklearn.model_selection import train_test_split

# Convert ZipDataset to numpy arrays
paired_dataset_array = np.array(list(paired_dataset.as_numpy_iterator()))
paired_labels_dataset_array = np.array(list(paired_labels_dataset.as_numpy_iterator()))
# Split ratio for train-test split
split_ratio = 0.8

# Split the paired dataset and labels dataset into train and test
train_dataset, test_dataset, train_labels_dataset, test_labels_dataset = train_test_split(
    paired_dataset_array, paired_labels_dataset_array, test_size=1-split_ratio)

# Print the sizes of train and test datasets
print('Train Dataset Size:', len(train_dataset))
print('Train Labels Dataset Size:', len(train_labels_dataset))
print('Test Dataset Size:', len(test_dataset))
print('Test Labels Dataset Size:', len(test_labels_dataset))

In [ ]:
train_dataset[0]

In [ ]:
for i in range(len(train_dataset) // 2):
    train_dataset[i][0], train_dataset[i][1] = train_dataset[i][1].copy(), train_dataset[i][0].copy()
    train_labels_dataset[i][0], train_labels_dataset[i][1] = train_labels_dataset[i][1].copy(), train_labels_dataset[i][0].copy()

for i in range(len(test_dataset) // 2):
    test_dataset[i][0], test_dataset[i][1] = test_dataset[i][1].copy(), test_dataset[i][0].copy()
    test_labels_dataset[i][0], test_labels_dataset[i][1] = test_labels_dataset[i][1].copy(), test_labels_dataset[i][0].copy()

In [ ]:
# Generate shuffled indices
shuffled_indices = np.random.permutation(len(train_dataset))

# Shuffle train_dataset and train_labels_dataset using the shuffled indices
shuffled_train_dataset = train_dataset[shuffled_indices]
shuffled_train_labels_dataset = train_labels_dataset[shuffled_indices]

# Print the sizes of shuffled train dataset and labels dataset
print('Shuffled Train Dataset Size:', len(shuffled_train_dataset))
print('Shuffled Train Labels Dataset Size:', len(shuffled_train_labels_dataset))
print(shuffled_indices)

In [ ]:
# Generate shuffled indices
shuffled_indices = np.random.permutation(len(test_dataset))

# Shuffle train_dataset and train_labels_dataset using the shuffled indices
shuffled_test_dataset = test_dataset[shuffled_indices]
shuffled_test_labels_dataset = test_labels_dataset[shuffled_indices]

# Print the sizes of shuffled train dataset and labels dataset
print('Shuffled Train Dataset Size:', len(shuffled_test_dataset))
print('Shuffled Train Labels Dataset Size:', len(shuffled_test_labels_dataset))

In [ ]:
def scheduler(epoch, lr):
    return lr * 0.99

# Create the learning rate scheduler callback
lr_scheduler = callbacks.LearningRateScheduler(scheduler)

# Set the initial learning rate
initial_learning_rate = 0.01

In [ ]:
# Extract inputs from the paired_dataset_array
paired_dataset_input1 = shuffled_train_dataset[:, 0]
paired_dataset_input2 = shuffled_train_dataset[:, 1]


In [ ]:
# Extract inputs from the paired_dataset_array
paired_dataset_input1 = shuffled_train_dataset[:, 0]
paired_dataset_input2 = shuffled_train_dataset[:, 1]

# Create and compile the Siamese model
siamese_model.compile(optimizer=tf.keras.optimizers.Adamax(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
siamese_model.fit([paired_dataset_input1, paired_dataset_input2], shuffled_train_labels_dataset, epochs=1000, batch_size=1000, callbacks = [lr_scheduler])

In [ ]:
# Evaluate the Siamese model on the test dataset
test_loss, test_accuracy = siamese_model.evaluate([shuffled_test_dataset[:, 0], shuffled_test_dataset[:, 1]], shuffled_test_labels_dataset)

# Print the accuracy
print('Test Accuracy:', test_accuracy*100)

In [ ]:
paired_dataset_input1[0]

In [ ]:
def preprocess_for_siamese_input(fen):
    bitstring = np.array([fen2bitstring(fen)])
    siamese_input = [bitstring, bitstring]
    return siamese_input

In [ ]:
fen = 'rn3b1r/pp4pp/2pk1p2/5B2/8/1P2PP2/P1P3PP/RNB1K2R b KQ - 0 13'
x = preprocess_for_siamese_input(fen)

In [ ]:
siamese_model.predict(x)

In [ ]:
siamese_model.save("unsupervised_siamese_100000-300000_V2.h5")